In [50]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#MongoDB Module
from PythonModule import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

# Connect to database via Python CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
# Retriving all Dog records
df = pd.DataFrame.from_records(db.read({"animal_type": "Dog"}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id','datetime','monthyear','age'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the layout of the Dash app
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Header section with an image, title, and author name
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
        height=100, width=100), 
        html.H1('SNHU CS-340 Dashboard', style={'marginLeft': '20px'}),
        html.Br(),
        html.H2('by Harshilkumar Jayswal', style={'marginLeft': '10px'})
        ], style={'display': 'flex', 'alignItems':'center', 'justifyContent':'center'}),
    
    html.Hr(), # Horizontal line for visual separation

    html.Div([
       # Radio buttons for filtering rescue dog types   
        dcc.RadioItems(
        id='data_filter', # ID for callback function
        options=[
        {'label': 'Water Rescue', 'value': 'Water'},
        {'label': 'Mountain Rescue','value': 'Mountain or Wilderness'},
        {'label': 'Disaster Rescue', 'value': 'Disaster or Individual Tracking'},
        {'label': 'Reset', 'value': 'None'},
        ],
        value = 'None', # Default value for the radio items
        inline=True # Display radio items inline
        )   
    ], style={'display': 'horizontal', 'alignItems':'left', 'justifyContent':'left'}),
    html.Hr(), # Horizontal line for visual separation
    
    # Create a DataTable component to display data in tabular format
    dash_table.DataTable(
        id='datatable-id', # Unique identifier for the DataTable component
        columns=[  # Define columns for the table based on DataFrame columns
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        # Convert DataFrame to dictionary of records for DataTable
        data=df.to_dict('records'),
        
        # Enable sorting
        sort_action='native',

        # Enable single row selection
        row_selectable='single',

        # Enable filtering
        filter_action='native',        

        # Enable pagination
        page_action='native',
        page_current=0,  # Set initial page number
        page_size=15,  # Set number of rows visible per page

        # Conditional formatting can be added here based on your requirements

        # Additional optional features for enhancing user interaction
        # Styling for the table header
        style_header={
            'backgroundColor': 'white',
            'fontWeight': 'bold'
        },
        
        # Allowing Horizontal scroll
        style_table={'overflowX': 'auto'},  
        
        # Allowing cell to fit certain length of characters
        style_cell={
           'minWidth': '180px', 'width': '120px', 'maxWidth': '180px', 
           'overflow': 'hidden',
            'textOverflow': 'ellipsis',
         },  
          
),
    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div(
                    dcc.Graph(id='pie-chart'),
                    className='col s12 m6',

            ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
            )
        ]),
    

])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Update the conditional styling of the datatable based on selected columns.
    if selected_columns is None:
        return [] #Return an empty list if none
    return [{
        'if': { 'column_id': i },
        } for i in selected_columns]


#This callback resets the selections on the data table to none when none is selected
@app.callback(
    Output('datatable-id', 'selected_rows'),  # Update the selected rows of the datatable
    [Input('data_filter', 'value')]  # Listen to changes in the value of the radio items
)
def reset_table_selection(selected_type):
    return [] #Always return a blank list so the selections reset every option selection


#This callback updates a pie chart of the animal types
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_breed_distribution_chart(data):
    
    # Update the breed distribution chart based on the provided data.
    if data is None or not data:
        return px.pie(title="No Data Available")
    
    df = pd.DataFrame(data)
    if 'breed' in df.columns:
        
        # Aggregate data to count the frequency of each breed
        breed_counts = df['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        
        # Create a pie chart
        fig = px.pie(
            df, 
            names='breed', 
            title='Preferred Animals',
            height=500,
            width=1000
        )  
        
        # Adjust the position of text labels inside the pie chart slices
        fig.update_traces(textposition='inside')
        
        # Customize the layout of the pie chart
        fig.update_layout(
            autosize=False,
            width=1000,
            height=500,
            margin=dict(l=0, r=0, t=30, b=0),
            xaxis_title="Number of Dogs",
            yaxis_title=None,  # Hides the axis title for breeds since it's evident from the data
            showlegend=True,
            uniformtext_minsize=12, 
            uniformtext_mode='hide'
        )
        return fig
    else:
        return px.pie(title="No Data Available for Breeds")



#This callback updates the data underlying the view based upon the animal type selections in the radio menu
@app.callback(
    Output('datatable-id', 'data'),  # Update the data of the datatable
    [Input('data_filter', 'value')]  # Listen to changes in the value of the radio items
)
def update_table(selected_type):
    if selected_type == 'None':
        query = {"animal_type": "Dog"}
        
    elif selected_type == 'Water':     
        query = {
            "animal_type": "Dog",
            "breed": {"$regex": "Labrador Retriever Mix|Chesapeake Bay Retriever|Newfoundland", "$options": "i"},
            "sex_upon_outcome":"Intact Female" ,
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}  
        }
        
    elif selected_type == 'Mountain or Wilderness':
        query = {
            "animal_type": "Dog",
            "breed": {"$regex": "German Shepherd|Alaskan Malamute|Siberian Husky|Rottweiler","$options": "i"},
            "sex_upon_outcome":"Intact Male" ,
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156} 
        }
        
    elif selected_type == 'Disaster or Individual Tracking':
        query = {
            "animal_type": "Dog",
            "breed": {"$regex": "Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler","$options": "i"},
            "sex_upon_outcome":"Intact Male" ,
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}  
        }
    else:
        query = {"animal_type": "Dog"}

    # Fetch new data from MongoDB based on the selected type
    new_df = pd.DataFrame.from_records(db.read(query))
    new_df.drop(columns=['_id','datetime','monthyear'], inplace=True)  # Removing the '_id' column
    
    return new_df.to_dict('records')  # Update the table's data

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, selected_rows):
    #default center and zoom
    center = [20,0]
    zoom = 2
    
    if viewData is None or not selected_rows:
        #Return default if no row is selected
        return [dl.Map(center=center, zoom=zoom, children=[dl.TileLayer()], style={'width':'1000px', 'height':'500px'})]        


    # This conversion step is necessary to access the selected row correctly
    rows = pd.DataFrame.from_records(viewData)  # Convert viewData to a DataFrame
    
    # Now, you can access the selected_row as shown
    if selected_rows and viewData and selected_rows[0] < len(rows):
        
        selected_row = rows.iloc[selected_rows[0]] #if row is selected row
        latitude = selected_row['location_lat']
        longitude = selected_row['location_long']
        center = [latitude, longitude] #center on selected row
        zoom = 10 #Zoom in to location
        
        #create a marker
        marker = dl.Marker(position=center, children=[
            dl.Tooltip(selected_row['breed']),
            dl.Popup(html.Div([
                html.B("Animal Name:"),
                html.Br(),
                html.Div(selected_row['name']) #animal name
            ]))
        ])
        
        #Updating the map's children to include the new marker
        return [dl.Map(center=center, zoom=zoom, children=[
            dl.TileLayer(),
            marker
        ], style={'width': '1000px', 'height': '500px'})]

app.run_server(debug=True)



Dash app running on http://127.0.0.1:15110/
